# Анализ клиентов страховой компании

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
insurance = pd.read_csv('/datasets/insurance.csv')

In [3]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
insurance.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [5]:
insurance.duplicated().sum()

153

In [6]:
insurance.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

данные загружены, таблица заполнена, пропусков нет, можно приступать к работе с матрицами.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

# Вопрос: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

Убедимся в том, что линейня регрессия из LinearRegression()вычисляет значения по формуле.

In [7]:
features = insurance.drop('Страховые выплаты',axis=1)
target = insurance['Страховые выплаты']

In [8]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))
model.coef_


0.4249455028666801


array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

 Теперь перейдем к теоретической части.

Обозначения для работы с уровнениями линейной регрессии:

 — матрица признаков (нулевой столбец состоит из единиц)

 — вектор целевого признака

 — матрица, на которую умножаются признаки

 — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

**Ответ:** не изменится.

**Обоснование:** Заменим матрицу X на матрицу Z:

Z=XP  (2.1),
где P - обратимая матрица с некими значениями, на которую может быть умножена матрица X.

Обоснование: 

Заменим X на Z и вычислим, чему будет равено предсказание и вектор весов.
$$
 a_1 =  Z w_1  (2.2)
$$


$$
w_1 = (Z^T Z)^{-1} Z^T y  (2.3)
$$
Подставим уравнение 2.2 правую часть уравнения 2.3 и получим следущее:
$$
a_1=Z (Z^T Z)^{-1} Z^T y  (2.4)
$$

Вспомним что Z = XP и заменим Z, получим:

$$
 a_1 = XP((XP)^T(XP))^{-1} (XP)^T  y    (2.5)
$$

Для дальнейшей работы нам понадобится следующие свойсва обратной матрицы:



$$
(AB)^T=B^T A^T \tag 1
$$
$$
(AB)^{-1} = B^{-1} A^{-1} \tag 2
$$
$$
A A^{-1} = A^{-1} A = E \tag 3
$$
$$
AE = EA = A \tag 4
$$
$$
A(BC) = (AB)C \tag 5
$$
тогда получим следующие выражение 
$$
a_1
$$
$$
a_1 = XP((XP)^T(XP))^{-1}(XP)^T y= XP (X^T P^T X P)^{-1}(XP)^T y=XP(P^T(X^TXP))^{-1}P^T X^Ty
$$
$$
a_1=XP(P^T(X^TX)P^){-1}P^TX^T y=XP((X^TX)P)^{-1} P^{T^{-1}}P^TX^T y
$$
Умножение 
$$
(P^T)^{-1} P^T = E (единичную матрицу),
$$
Посмотрим, что осталось от уравнения :
$P^{T^{-1}} P^T$ - сократим
$$
a_1=XP P^{-1}(X^TX)^{-1}P^{T^{-1}}P^TX^T y=XP P^{-1} (X^TX)^{-1}X^T y
$$

$P^{-1} P$ - сократим и вспомним что

$$
w=(X^TX)^{-1}X^T y
$$
получим: 
$$
a_1=Xw=a
$$
Как видно, значение предсказания a не меняется, если умножать матрицу признаков на обратимую матрицу.

## Алгоритм преобразования

**Алгоритм**

Для защиты информации на этапе поиска коэффициентов регрессии и предсказаний регрессии будем умножать матрицу признаков на обратимую матрицу Y, которая будет генерироваться случайным образом.

Этапы алгоритма:

1. Составление матрицы Y. 
2. Проверка матрицы на обратимость. Вычисление детерминанта матрицы  Y.
3. Получение матрицы преобразованных признаков  Z = XY.
4. Применение алгоритма на преобразованных признаках Z.



**Обоснование**

Матрица Y должна иметь необходимую размерность (nxn), где n - количество признаков для регрессии. Таким образом матрица Z будет иметь туже размерность, что и матрица X. Обратная матрица Y существует только для квадратных невырожденных матриц (определитель которых не равен нулю).

Пример: X=[[1,2],[2,3],[4,5]],

Y=[[1,0],[2,3]] detY=3

Найдём значение Z=XY=[[1,2],[2,3],[4,5]]*[[1,0],[2,3]]=[[1*1+2*2,1*0+2*3],[2*1+3*2,2*0+3*3],[4*1+5*2,4*0+5*3]]=[[5,6],[8,9],[14,15]]

После этого добавляем нулевой столбец и вводим данные в линейную регрессию.




## Проверка алгоритма

Проведем исследование модели по двум направлениям:

Исследуем качество модели без преобразования. 1.1 С исходными признаками 1.2 С отмасштабированными признаками
Исследуем качество модели с преобразованием. 2.1 С исходными признаками 2.2 С отмасштабированными признаками

Разделим данные на обучающие и тестовые.

In [9]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [10]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_origin_data = r2_score(target_test, model.predict(features_test))
print("w-vector coef",model.coef_)
print("R2 =", R2_LR_origin_data)

w-vector coef [ 1.79258369e-02  3.57228278e-02 -5.46000708e-07 -1.26186590e-02]
R2 = 0.43522757127026546


In [11]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_origin_data_scaled = r2_score(target_test, pipeline.predict(features_test))
#print("w-vector coef",pipeline.coef_) при использовании pipeline невозможно получить коэффициенты регрессии
print("R2 =", R2_LR_origin_data_scaled)

R2 = 0.4352275712702668


Модель ведет себя одинаково как на исходных данных, так и на отмасштабированных. Колебания 15 знака и дальше после запятой связано с точностью хранения дробных чисел в python 3.

Создадим функцию преобразования матрицы признаков.

In [12]:
def cipher_features(features):
    crypted_features = features
    n = features.shape[1]
    np.random.seed(7072020)
    cipher_matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(cipher_matrix)
    while det == 0:
        np.random.seed(7072021)
        cipher_matrix = np.random.randint(1, 10, (n,n))
        det = np.linalg.det(cipher_matrix)
    crypted_features = crypted_features @ cipher_matrix
    return crypted_features, cipher_matrix

In [13]:
display(features.head())
features, cipher_matrix = cipher_features(features)
display(features.head())
cipher_matrix

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


,0,1,2,3
0,397184.0,49981.0,99373.0,248097.0
1,304422.0,38421.0,76185.0,190101.0
2,168261.0,21261.0,42116.0,105058.0
3,333805.0,41903.0,83486.0,208560.0
4,209059.0,26357.0,52320.0,130562.0


array([[7, 5, 8, 6],
       [9, 9, 4, 2],
       [8, 1, 2, 5],
       [8, 7, 1, 9]])

Как видно, данные потеряли свои оригинальные значения, теперь в них хранится непонятное число. Разобьем данные на тренировочную и обучающую выборку и проверим R2 значение.

In [14]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [15]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_cipher_data = r2_score(target_test, model.predict(features_test))
print("w-vector coef",model.coef_)
print("R2 =", R2_LR_cipher_data)

w-vector coef [ 0.00205789  0.00163118  0.00304974 -0.00483886]
R2 = 0.43522757127025513


In [16]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_cipher_data_scaled = r2_score(target_test, pipeline.predict(features_test))
#print("w-vector coef",pipeline.coef_) при использовании pipeline невозможно получить коэффициенты регрессии
print("R2 =", R2_LR_cipher_data_scaled)

R2 = 0.4352275712702951


Сравним показатели качества моделей.

In [17]:
result = pd.DataFrame(data= [R2_LR_origin_data_scaled,
                      R2_LR_origin_data,
                      R2_LR_cipher_data,
                      R2_LR_cipher_data_scaled], 
                     columns=['R2'], 
                     index=['Линейная регрессия',
                            'Линейная регрессия c масштабом',
                            'Линейная регрессия на преобразованных признаках',
                            'Линейная регрессия на преобразованных признаках c масштабом',])
result

,R2
Линейная регрессия,0.435228
Линейная регрессия c масштабом,0.435228
Линейная регрессия на преобразованных признаках,0.435228
Линейная регрессия на преобразованных признаках c масштабом,0.435228


Итоговый вывод
В ходе работы было проделано:

Загружены и изучены данные.
Качество линейной регресии не изменилось от использования исходной матрицы и исходной матрицы, умноженную на обратимую.
Создан алгоритм который преобразовывает данные.
Исследован алгоритм преобразования данных и проверена метрика R2 для данных без преобразования и с ним.
По результатам работы использования матричных операций можно увидеть, что данные очень просто зашифровать от распознования, имея правильную матрицу.